In [ ]:
# import argparse
import os
import numpy as np
import random
import math
import itertools

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch
cuda = True if torch.cuda.is_available() else False

# Set random seed for reproducibility
# manualSeed = 999
manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
np.random.seed(manualSeed)
torch.manual_seed(manualSeed)


Random Seed:  3613


In [ ]:
# parser = argparse.ArgumentParser()
# parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")
# parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
# parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
# parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
# parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
# parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
# parser.add_argument("--latent_dim", type=int, default=62, help="dimensionality of the latent space")
# parser.add_argument("--img_size", type=int, default=32, help="size of each image dimension")
# parser.add_argument("--channels", type=int, default=1, help="number of image channels")
# parser.add_argument("--sample_interval", type=int, default=400, help="interval between image sampling")
# opt = parser.parse_args()

class Object(object):
    pass

opt = Object()
opt.n_epochs = 10000 # number of epochs of training
opt.batch_size = 64 # size of the batches
opt.lr = 0.0002 # adam: learning rate
opt.b1 = 0.5 # adam: decay of first order momentum of gradient
opt.b2 = 0.997 # adam: decay of first order momentum of gradient
opt.n_cpu = 8 # number of cpu threads to use during batch generation
opt.latent_dim = 50 # dimensionality of the latent space
opt.img_size = 64 # size of each image dimension
# opt.img_size = 32 # size of each image dimension
opt.channels = 3 # number of image channels
opt.sample_interval = 100 # interval between image sampling
img_shape = (opt.channels, opt.img_size, opt.img_size)

In [ ]:
imgDir = 'emoji'
# Configure data loader
os.makedirs(imgDir, exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.ImageFolder(root=imgDir,
      transform=transforms.Compose([
          transforms.Resize(opt.img_size),
          transforms.CenterCrop(opt.img_size),
          transforms.ToTensor(),
          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
      ])),
    batch_size=opt.batch_size,
    shuffle=True,
)

# Prepare folder for storing results
os.makedirs("images", exist_ok=True)

def sample_image(n_row, batches_done):
    """Saves a grid of generated digits"""
    # Sample noise
    z = Variable(Tensor(np.random.normal(0, 1, (n_row ** 2, opt.latent_dim))))
    gen_imgs = decoder(z)
    save_image(gen_imgs.data, "images/%d.png" % batches_done, nrow=n_row, normalize=True)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        # self.l1, self.conv_blocks = self.conv_blocks1()
        self.l1, self.conv_blocks = self.conv_blocks2()
          
    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], self.layer1_depth, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img
      
    def conv_blocks1(self):
        # ----------------------
        # adjustable parameters
        self.init_size = opt.img_size // 4
        self.layer1_depth = 32
        self.layer2_depth = 16
        self.layer3_depth = 16
        self.layer4_depth = 16
        # ----------------------

        l1 = nn.Sequential(nn.Linear(opt.latent_dim, self.layer1_depth * self.init_size ** 2))
        # output dim: (self.layer1_depth, self.init_size, self.init_size)
        return l1, nn.Sequential(
            # Layer 1
            nn.Upsample(scale_factor=2, mode='bicubic'),
            nn.LeakyReLU(0.8, inplace=True),
            # output dim: (self.layer1_depth, scale_factor * self.init_size, scale_factor * self.init_size)
            nn.Conv2d(self.layer1_depth, self.layer2_depth, 3, stride=1, padding=1),
            # output dim: (self.layer2_depth, h, w)
            # where h=w=(scale_factor * self.init_size)

            # Layer 2
            nn.BatchNorm2d(self.layer2_depth, 0.8),
            nn.LeakyReLU(0.8, inplace=True),
            nn.Upsample(scale_factor=2, mode='bicubic'),
            # output dim: (self.layer2_depth, 2*h, 2*w)
            nn.Conv2d(self.layer2_depth, self.layer3_depth, 3, stride=1, padding=1),
            # output dim: (self.layer3_depth, 2*h, 2*w)

            # Layer 3
            nn.BatchNorm2d(self.layer3_depth, 0.8),
            nn.LeakyReLU(0.8, inplace=True),
            nn.Conv2d(self.layer3_depth, self.layer4_depth, 3, stride=1, padding=1),
            # output dim: (self.layer4_depth, 2*h, 2*w)

            # Layer 4
            nn.BatchNorm2d(self.layer4_depth, 0.8),
            nn.LeakyReLU(0.8, inplace=True),
            nn.Conv2d(self.layer4_depth, opt.channels, 3, stride=1, padding=1),
            # output dim: (opt.channels, 2*h, 2*w)

            # Output Layer
            nn.Tanh(),
            #nn.Sigmoid(),
        )

    def conv_blocks2(self):
        # ----------------------
        # adjustable parameters
        self.init_size = opt.img_size // 4
        self.layer1_depth = 1024
        self.layer2_depth = 512
        self.layer3_depth = 256
        self.layer4_depth = 16
        self.layer5_depth = 64
        # ----------------------

        l1 = nn.Sequential(nn.Linear(opt.latent_dim, self.layer1_depth * self.init_size ** 2))
        # output dim: (self.layer1_depth, self.init_size, self.init_size)
        return l1, nn.Sequential(
            # Layer 1
            nn.Dropout2d(),
            #nn.BatchNorm2d(self.layer1_depth, 0.8),
            # nn.LeakyReLU(0.2, inplace=True),
            nn.ReLU(),
            nn.ConvTranspose2d(self.layer1_depth, self.layer2_depth, 4, stride=2, padding=1),
            # output dim: (self.layer2_depth, h, w)
            # where h=w=(self.init_size-1)*stride-2*padding+(kernel_size-1)+1

            # Layer 2
            nn.Dropout2d(),
            nn.BatchNorm2d(self.layer2_depth, 0.8),
            # nn.LeakyReLU(0.2, inplace=True),
            nn.ReLU(),
            nn.ConvTranspose2d(self.layer2_depth, self.layer3_depth, 4, stride=2, padding=1, bias=False),
            # nn.ConvTranspose2d(self.layer2_depth, opt.channels, 4, stride=2, padding=1),
            # output dim: (self.layer3_depth, hh, ww)
            # where hh=ww=(h-1)*stride-2*padding+(kernel_size-1)+1


            # Layer 3
            nn.Dropout2d(),
            nn.BatchNorm2d(self.layer3_depth, 0.8),
            # nn.LeakyReLU(0.2, inplace=True),
            nn.ReLU(),
            nn.Conv2d(self.layer3_depth, self.layer4_depth, 3, stride=1, padding=1, bias=False),
            # nn.ConvTranspose2d(self.layer3_depth, opt.channels, 1, stride=1, padding=0, bias=False),
            # output dim: (self.layer4_depth, hh, ww)

            # Layer 4
            nn.Dropout2d(),
            nn.BatchNorm2d(self.layer4_depth, 0.8),
            # nn.LeakyReLU(0.2, inplace=True),
            nn.ReLU(),
            nn.ConvTranspose2d(self.layer4_depth, opt.channels, 1, stride=1, padding=0, bias=False),
            # output dim: (opt.channels, hh, ww)

            # # Layer 5
            # nn.BatchNorm2d(self.layer5_depth, 0.8),
            # nn.LeakyReLU(0.2, inplace=True),
            # nn.ConvTranspose2d(self.layer5_depth, opt.channels, 1, stride=1, padding=0),
            # # output dim: (self.layer5_depth, h, w)

            # Output Layer
            nn.Tanh(),
            #nn.Sigmoid(),
        )

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(opt.channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = opt.img_size // 2 ** 4
        # print('ds_size: {}'.format(ds_size))
        self.adv_layer = nn.Linear(128 * ds_size ** 2, 1)

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.05)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)



In [ ]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

In [ ]:

# !!! Minimizes MSE instead of BCE
adversarial_loss = torch.nn.MSELoss()
# adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
print(generator.conv_blocks)
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()


# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Sequential(
  (0): Dropout2d(p=0.5, inplace=False)
  (1): ReLU()
  (2): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (3): Dropout2d(p=0.5, inplace=False)
  (4): BatchNorm2d(512, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (7): Dropout2d(p=0.5, inplace=False)
  (8): BatchNorm2d(256, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (11): Dropout2d(p=0.5, inplace=False)
  (12): BatchNorm2d(16, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  (13): ReLU()
  (14): ConvTranspose2d(16, 3, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (15): Tanh()
)


Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout2d(p=0.25, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout2d(p=0.25, inplace=False)
    (6): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): LeakyReLU(negative_slope=0.2, inplace=True)
    (13): Dropout2d(p=0.25, inplace=False)
    (14): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  )
  (adv_laye

In [ ]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

In [ ]:
import matplotlib.pylab as plt
def plot_loss(loss_G, loss_D):
    plt.plot(loss_G, 'b',label='Generator Loss ')
    plt.plot(loss_D,'r',label='Discriminator Loss')
    plt.xlabel('iteration')
    plt.title("loss")
    plt.legend()
    plt.show()

In [ ]:
loss_D , loss_G = [], []
for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        # lll = torch.nn.BCELoss()
        # g_loss = lll(discriminator(gen_imgs), valid)
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = 0.5 * (real_loss + fake_loss)

        # adaptive discriminator weight update
        # if d_loss.item()/g_loss.item() > 0.1:
        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch+1, opt.n_epochs, i+1, len(dataloader), d_loss.item(), g_loss.item())
        )
        loss_G.append(g_loss.item())
        loss_D.append(d_loss.item())
        batches_done = epoch * len(dataloader) + i + 1
        if batches_done % opt.sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True, scale_each=True)
            # save_image(real_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)
            # plot(loss_G, loss_D)

[Epoch 1/10000] [Batch 1/1] [D loss: 0.086269] [G loss: 0.869390]
[Epoch 2/10000] [Batch 1/1] [D loss: 0.110899] [G loss: 1.002932]
[Epoch 3/10000] [Batch 1/1] [D loss: 0.059283] [G loss: 0.674251]
[Epoch 4/10000] [Batch 1/1] [D loss: 0.111647] [G loss: 0.375694]
[Epoch 5/10000] [Batch 1/1] [D loss: 0.058991] [G loss: 0.560522]
[Epoch 6/10000] [Batch 1/1] [D loss: 0.094429] [G loss: 1.015013]
[Epoch 7/10000] [Batch 1/1] [D loss: 0.052606] [G loss: 0.387064]
[Epoch 8/10000] [Batch 1/1] [D loss: 0.109645] [G loss: 0.730059]
[Epoch 9/10000] [Batch 1/1] [D loss: 0.091342] [G loss: 0.733719]
[Epoch 10/10000] [Batch 1/1] [D loss: 0.139500] [G loss: 0.752677]
[Epoch 11/10000] [Batch 1/1] [D loss: 0.156158] [G loss: 0.441411]
[Epoch 12/10000] [Batch 1/1] [D loss: 0.117574] [G loss: 0.761698]
[Epoch 13/10000] [Batch 1/1] [D loss: 0.083392] [G loss: 0.889651]
[Epoch 14/10000] [Batch 1/1] [D loss: 0.057743] [G loss: 0.522150]
[Epoch 15/10000] [Batch 1/1] [D loss: 0.066269] [G loss: 0.280714]
[Epo

In [ ]:
torch.save(generator,'DCGAN.pt')

In [ ]:
print(real_imgs.size())
print(gen_imgs.size())